In [2]:
import bs4
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup 
from collections import defaultdict

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def scroll_page(URL):
    SCROLL_PAUSE_TIME = 3
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(URL)
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    i = 0
    while i<300:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        i = i + 1
        last_height = new_height
    return driver.page_source


In [4]:
def crawlHTML(htmlText):    
    soup = BeautifulSoup(htmlText, "html.parser")
    i=0
    dic = {}
    list1 = []
    for t in soup.findAll(class_="tableTitles"):
        if i == 1:
            break
        i = i + 1
        for f in t.findAll("button"):            
            dic[f["aria-label"]] = []
    df=pd.DataFrame(dic)
    for t in soup.findAll(class_="tableRow"):
        for f in t.findAll(class_="tableCol"):
            k = f.find("div")             
            if k is not None:
                list1.append(k.text.strip())
            else:
                list1.append("")
        list1.pop()
        df.loc[len(df)] = list1
        list1 = []
    return df

Drop irrelevant columns

In [5]:
def drop_col_from_df(df):
    df.drop(columns =['','מגמת שינוי','גוש חלקה - תת חלקה','סוג נכס'], axis = 1, inplace = True)

In [6]:
def save_csv(df,name):
    headerList = ['Date', 'Address', 'Rooms', 'Floor', 'Square', 'Price']
    df.to_csv(name,header=headerList, index=False, encoding = "ISO-8859-8")

In [ ]:
UrlRovaBet = "https://www.nadlan.gov.il/?search=%D7%A8%D7%95%D7%91%D7%A2%20%D7%91%20%D7%90%D7%A9%D7%93%D7%95%D7%93"
UrlRovaDaled = "https://www.nadlan.gov.il/?search=%D7%90%D7%A9%D7%93%D7%95%D7%93%20%D7%A8%D7%95%D7%91%D7%A2%20%D7%93"
UrlRovaYudAlef = "https://www.nadlan.gov.il/?search=%D7%90%D7%A9%D7%93%D7%95%D7%93%20%D7%A8%D7%95%D7%91%D7%A2%20%D7%99%22%D7%90"
UrlRovaVav = "https://www.nadlan.gov.il/?search=%D7%90%D7%A9%D7%93%D7%95%D7%93%20%D7%A8%D7%95%D7%91%D7%A2%20%D7%95"

listOfUrl = [UrlRovaBet,UrlRovaDaled,UrlRovaYudAlef,UrlRovaVav]
listNameOfCsv = ["rovaBet.csv","rovaDaled.csv","rovaYudAlef.csv", "rovaVav.csv"]

for url,name in zip(listOfUrl,listNameOfCsv):    
    html = scroll_page(url)
    df = crawlHTML(html)
    drop_col_from_df(df)
    save_csv(df,name)